In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,15.26,79,1,7.29,AU,1712501963
1,1,nar'yan-mar,67.6713,53.0870,-5.50,88,94,6.26,RU,1712501997
2,2,ribeira grande,38.5167,-28.7000,18.76,88,40,9.77,PT,1712501974
3,3,adamstown,-25.0660,-130.1015,22.69,76,38,4.76,PN,1712501990
4,4,san antonio de pale,-1.4014,5.6325,29.05,75,79,4.28,GQ,1712501985


In [3]:
%%capture --no-display

# Configure the map plot
city_plt_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    c="City",
    s= "Humidity",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
city_plt_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_df.dropna(how = "any")

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,san juan,-31.5375,-68.5364,26.97,47,0,3.75,AR,1712501950
182,182,pimentel,-6.8367,-79.9342,25.16,69,0,1.54,PE,1712502143
229,229,illapel,-31.6308,-71.1653,22.35,37,0,2.03,CL,1712502097
252,252,dolores,-36.3132,-57.6792,24.16,48,0,4.36,AR,1712502159
277,277,saint augustine beach,29.8505,-81.2654,21.72,52,0,3.60,US,1712502165
319,319,tagta,41.6504,59.9164,22.97,28,0,3.09,TM,1712502175
377,377,cape canaveral,28.4058,-80.6048,23.90,53,0,3.60,US,1712502194
439,439,colonia,50.9333,6.9500,24.42,45,0,2.57,DE,1712502008
456,456,acapulco de juarez,16.8634,-99.8901,26.07,51,0,1.38,MX,1712501984
485,485,baldivis,-32.3299,115.8218,23.40,56,0,4.47,AU,1712502220


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
113,san juan,AR,-31.5375,-68.5364,47,
182,pimentel,PE,-6.8367,-79.9342,69,
229,illapel,CL,-31.6308,-71.1653,37,
252,dolores,AR,-36.3132,-57.6792,48,
277,saint augustine beach,US,29.8505,-81.2654,52,
319,tagta,TM,41.6504,59.9164,28,
377,cape canaveral,US,28.4058,-80.6048,53,
439,colonia,DE,50.9333,6.9500,45,
456,acapulco de juarez,MX,16.8634,-99.8901,51,
485,baldivis,AU,-32.3299,115.8218,56,


In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey":geoapify_key,
    "format": "json"
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san juan - nearest hotel: Hotel San Francisco
pimentel - nearest hotel: Hospedaje Calle del Sol
illapel - nearest hotel: Domingo Ortiz de Rosas
dolores - nearest hotel: Plaza Hotel
saint augustine beach - nearest hotel: Castillo Real Resort Hotel
tagta - nearest hotel: No hotel found
cape canaveral - nearest hotel: Holiday Inn Club Vacations Cape Canaveral Beach Resort
colonia - nearest hotel: Wasserturm Hotel Cologne
acapulco de juarez - nearest hotel: Hotel del Valle
baldivis - nearest hotel: No hotel found
zaragoza - nearest hotel: Hotel Pilar Plaza


,City,Country,Lat,Lng,Humidity,Hotel Name
113,san juan,AR,-31.5375,-68.5364,47,Hotel San Francisco
182,pimentel,PE,-6.8367,-79.9342,69,Hospedaje Calle del Sol
229,illapel,CL,-31.6308,-71.1653,37,Domingo Ortiz de Rosas
252,dolores,AR,-36.3132,-57.6792,48,Plaza Hotel
277,saint augustine beach,US,29.8505,-81.2654,52,Castillo Real Resort Hotel
319,tagta,TM,41.6504,59.9164,28,No hotel found
377,cape canaveral,US,28.4058,-80.6048,53,Holiday Inn Club Vacations Cape Canaveral Beac...
439,colonia,DE,50.9333,6.9500,45,Wasserturm Hotel Cologne
456,acapulco de juarez,MX,16.8634,-99.8901,51,Hotel del Valle
485,baldivis,AU,-32.3299,115.8218,56,No hotel found


In [7]:
%%capture --no-display

# Configure the map plot
hotel_plt_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    c="City",
    frame_width = 800,
    frame_height = 600,
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotel_plt_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)